In [1]:
import re

def generate_plural(word):
    if re.search(r'(s|x|z|ch|sh)$', word):
        return word + 'es', 'E-insertion'
    elif re.search(r'[^aeiou]y$', word):
        return word[:-1] + 'ies', 'Y-replacement'
    else:
        return word + 's', 'S-addition'


def analyze_word(word, known_nouns):
    if word in known_nouns:
        return f"{word}+N+SG"

    # Try rule 1: E-insertion
    if word.endswith("es"):
        stem = word[:-2]
        if re.search(r'(s|x|z|ch|sh)$', stem) and stem in known_nouns:
            return f"{stem}+N+PL"

    # Try rule 2: Y-replacement
    if word.endswith("ies"):
        stem = word[:-3] + "y"
        if stem in known_nouns and re.search(r'[^aeiou]y$', stem):
            return f"{stem}+N+PL"

    # Try rule 3: S-addition
    if word.endswith("s"):
        stem = word[:-1]
        if stem in known_nouns and not re.search(r'(s|x|z|ch|sh|[^aeiou]y)$', stem):
            return f"{stem}+N+PL"

    return "Invalid Word"


# Load known nouns (assume singulars)
with open("brown_nouns.txt", "r") as f:
    known_nouns = set(word.strip().lower() for word in f)

# Example input list (replace with actual test)
test_words = ["foxes", "fox", "tries", "try", "bags", "bag", "foxs", "trys", "watchs"]

for word in test_words:
    print(f"{word} = {analyze_word(word.lower(), known_nouns)}")


foxes = fox+N+PL
fox = fox+N+SG
tries = tries+N+SG
try = try+N+SG
bags = bags+N+SG
bag = bag+N+SG
foxs = Invalid Word
trys = Invalid Word
watchs = Invalid Word


In [2]:
# FST for English noun pluralization

def pluralize(noun):
    # Define endings that require 'es'
    es_endings = ('s', 'z', 'x', 'ch', 'sh')
    # Check for singular
    if noun.endswith('s') or noun.endswith('es'):
        if noun.endswith('es'):
            stem = noun[:-2]
            if stem.endswith(es_endings):
                return f"{stem}+N+PL"
            else:
                return "Invalid Word"
        else:
            return "Invalid Word"
    elif noun.endswith('ies'):
        # Y replacement
        stem = noun[:-3]
        if stem.endswith('y'):
            return f"{stem}+N+PL"
        else:
            return "Invalid Word"
    elif noun.endswith('s'):
        # Regular plural
        stem = noun[:-1]
        if stem.endswith(es_endings):
            # Should be 'es' not just 's'
            return "Invalid Word"
        elif stem.endswith('y'):
            # Should be 'ies' not just 's'
            return "Invalid Word"
        else:
            return f"{stem}+N+PL"
    else:
        # Singular noun
        return f"{noun}+N+SG"

# Examples
words = ["fox", "foxes", "foxs", "watch", "watches", "bag", "bags", "try", "tries", "trys"]

for word in words:
    print(f"{word} = {pluralize(word)}")

fox = fox+N+SG
foxes = fox+N+PL
foxs = Invalid Word
watch = watch+N+SG
watches = watch+N+PL
bag = bag+N+SG
bags = Invalid Word
try = try+N+SG
tries = Invalid Word
trys = Invalid Word


In [3]:
# FST for English noun pluralization (correct Y-replacement handling)

def pluralize(noun):
    es_endings = ('s', 'z', 'x', 'ch', 'sh')

    # Plural: "ies" ending (Y-replacement rule)
    if noun.endswith('ies'):
        stem = noun[:-3]
        # The singular should be stem + "y"
        return f"{stem}y+N+PL"

    # Plural: "es" ending (E-insertion rule)
    elif noun.endswith('es'):
        stem = noun[:-2]
        # If the stem ends with an ES-ending, it's valid
        if stem.endswith(es_endings):
            return f"{stem}+N+PL"
        else:
            return "Invalid Word"

    # Plural: regular "s" ending (S addition rule)
    elif noun.endswith('s'):
        stem = noun[:-1]
        # If the stem ends with an ES-ending, it should have 'es', not just 's'
        if stem.endswith(es_endings):
            return "Invalid Word"
        # If the stem ends with 'y', it should have 'ies', not just 's'
        if stem.endswith('y'):
            return "Invalid Word"
        return f"{stem}+N+PL"

    # Singular noun
    else:
        return f"{noun}+N+SG"

# Examples
words = ["fox", "foxes", "foxs", "watch", "watches", "bag", "bags", "try", "tries", "trys"]

for word in words:
    print(f"{word} = {pluralize(word)}")

fox = fox+N+SG
foxes = fox+N+PL
foxs = Invalid Word
watch = watch+N+SG
watches = watch+N+PL
bag = bag+N+SG
bags = bag+N+PL
try = try+N+SG
tries = try+N+PL
trys = Invalid Word


In [5]:
# Pluralizer for all nouns in brown_noun.txt

def pluralize(noun):
    es_endings = ('s', 'z', 'x', 'ch', 'sh')

    # Plural: "ies" ending (Y-replacement rule)
    if noun.endswith('ies'):
        stem = noun[:-3]
        return f"{stem}y+N+PL"

    # Plural: "es" ending (E-insertion rule)
    elif noun.endswith('es'):
        stem = noun[:-2]
        if stem.endswith(es_endings):
            return f"{stem}+N+PL"
        else:
            return "Invalid Word"

    # Plural: regular "s" ending (S addition rule)
    elif noun.endswith('s'):
        stem = noun[:-1]
        if stem.endswith(es_endings):
            return "Invalid Word"
        if stem.endswith('y'):
            return "Invalid Word"
        return f"{stem}+N+PL"

    # Singular noun
    else:
        return f"{noun}+N+SG"


def process_file(input_file, output_file):
    with open(input_file, 'r') as infile, open(output_file, 'w') as outfile:
        for line in infile:
            noun = line.strip()
            if not noun:
                continue
            result = pluralize(noun)
            outfile.write(f"{noun} = {result}\n")


if __name__ == "__main__":
    process_file("brown_nouns.txt", "brown_noun_output.txt")

In [6]:
# Pluralizer for all nouns in brown_noun.txt

def pluralize(noun):
    es_endings = ('s', 'z', 'x', 'ch', 'sh')

    # Plural: "ies" ending (Y-replacement rule)
    if noun.endswith('ies'):
        stem = noun[:-3]
        # Singular form would be stem + "y"
        return f"{stem}y+N+PL"

    # Plural: "es" ending (E-insertion rule)
    elif noun.endswith('es'):
        stem = noun[:-2]
        # Accept only if stem ends with ES-ending
        if stem.endswith(es_endings):
            return f"{stem}+N+PL"
        else:
            return "Invalid Word"

    # Plural: regular "s" ending (S addition rule)
    elif noun.endswith('s'):
        # Avoid double-handling "ies" and "es"
        if noun.endswith('ies') or noun.endswith('es'):
            pass  # Already handled above
        stem = noun[:-1]
        # If stem ends with 'y', plural should be 'ies'
        if stem.endswith('y'):
            return "Invalid Word"
        # If stem ends with ES-ending, plural should be 'es'
        if stem.endswith(es_endings):
            return "Invalid Word"
        # Otherwise, valid plural (candidates, signatures, bags, etc.)
        return f"{stem}+N+PL"

    # Singular noun
    else:
        return f"{noun}+N+SG"


def process_file(input_file, output_file):
    with open(input_file, 'r') as infile, open(output_file, 'w') as outfile:
        for line in infile:
            noun = line.strip()
            if not noun:
                continue
            result = pluralize(noun)
            outfile.write(f"{noun} = {result}\n")


if __name__ == "__main__":
    process_file("brown_nouns.txt", "brown_noun_output.txt")

In [7]:
def pluralize(noun):
    es_endings = ('s', 'z', 'x', 'ch', 'sh')

    # Plural: "ies" ending (Y-replacement rule)
    if noun.endswith('ies'):
        stem = noun[:-3]
        return f"{stem}y+N+PL"

    # Plural: "es" ending (E-insertion rule)
    elif noun.endswith('es'):
        stem = noun[:-2]
        if stem.endswith(es_endings):
            return f"{stem}+N+PL"
        else:
            return "Invalid Word"

    # Plural: regular "s" ending (S addition rule)
    elif noun.endswith('s'):
        # Avoid double-handling "ies" and "es"
        if noun.endswith('ies') or noun.endswith('es'):
            return "Invalid Word"

        stem = noun[:-1]
        # Accept as regular plural
        return f"{stem}+N+PL"

    # Singular noun
    else:
        return f"{noun}+N+SG"


def process_file(input_file, output_file):
    with open(input_file, 'r') as infile, open(output_file, 'w') as outfile:
        for line in infile:
            noun = line.strip()
            if not noun:
                continue
            result = pluralize(noun)
            outfile.write(f"{noun} = {result}\n")


if __name__ == "__main__":
    process_file("brown_nouns.txt", "output.txt")


In [5]:
import string

class NounFST:
    """
    A Finite State Transducer (FST) to generate morphological features for English nouns.
    This FST handles three common pluralization rules:
    1. S-addition (e.g., bag -> bags)
    2. E-insertion after sibilants (e.g., fox -> foxes)
    3. Y-replacement (e.g., try -> tries)
    """

    def __init__(self):
        # Define the character sets for easier rule creation
        self.alphabet = set(string.ascii_lowercase)
        self.sibilants_1 = {'s', 'x', 'z'}
        self.sibilants_2_starters = {'c', 's'} # For 'ch' and 'sh'
        
        # All other non-special consonants and vowels. 'i' is handled specially.
        self.nonspecial_chars = self.alphabet - self.sibilants_1 - self.sibilants_2_starters - {'y', 'h', 'i'}

        # Define the states of our FST
        self.states = {
            "q_start",      # Initial state
            "q_root",       # Generic state for processing the root of a word
            "q_s_ending",   # State after encountering an 's' (could be 'sh' or plural)
            "q_c_ending",   # State after encountering a 'c' (could be 'ch')
            "q_y_ending",   # State after encountering a 'y'
            "q_sibilant",   # State for roots ending in a sibilant sound (s, x, z, ch, sh)
            "q_e_plural",   # State after seeing 'e' in an '-es' plural
            "q_i_plural",   # State after seeing 'i' in an '-ies' plural
            "q_accept_pl",  # Final accepting state for plurals
        }

        # The core of the FST: the transition table
        # Format: transitions[current_state][input_char] = (next_state, output_string)
        self.transitions = self._create_transitions()

    def _create_transitions(self):
        """Builds the transition dictionary."""
        transitions = {state: {} for state in self.states}
        
        # --- Transitions from q_start (and q_root, as they are similar) ---
        for state in ["q_start", "q_root"]:
            # Rule: Nonspecial characters keep us in the root state
            for char in self.nonspecial_chars:
                transitions[state][char] = ("q_root", char)
            # Rule: Sibilants move to the sibilant state
            for char in self.sibilants_1:
                transitions[state][char] = ("q_sibilant", char)
            # Rule: 'y' moves to the y-ending state
            transitions[state]['y'] = ("q_y_ending", 'y')
            # Rule: 'c' and 's' are special and need their own states to check for 'h'
            transitions[state]['c'] = ("q_c_ending", 'c')
            transitions[state]['s'] = ("q_s_ending", 's')

        # --- Transitions from specific ending states ---
        
        # State after seeing 'c'
        transitions["q_c_ending"]['h'] = ("q_sibilant", 'h') # Forms 'ch'
        for char in self.alphabet - {'h'}: # Any other character
             # FIXED: Changed self.transitions to the local 'transitions' dictionary
             transitions["q_c_ending"][char] = transitions["q_root"].get(char, (None, None))

        # State after seeing 's'
        transitions["q_s_ending"]['h'] = ("q_sibilant", 'h') # Forms 'sh'
        transitions["q_s_ending"]['e'] = ("q_e_plural", "") # Plural of s-ending root (bus -> buses)
        for char in self.alphabet - {'h', 'e'}: # Any other character
             # FIXED: Changed self.transitions to the local 'transitions' dictionary
             transitions["q_s_ending"][char] = transitions["q_root"].get(char, (None, None))

        # State for sibilant-ending roots (e.g., fox, watch)
        transitions["q_sibilant"]['e'] = ("q_e_plural", "") # E-insertion rule (fox -> foxes)

        # State for y-ending roots (e.g., try, boy)
        transitions["q_y_ending"]['i'] = ("q_i_plural", "y") # Y-replacement rule (tries)
        transitions["q_y_ending"]['s'] = ("q_accept_pl", "+N+PL") # Simple plural (boys)

        # States for handling plural endings
        transitions["q_i_plural"]['e'] = ("q_e_plural", "") # Middle of '-ies'
        transitions["q_e_plural"]['s'] = ("q_accept_pl", "+N+PL") # Final 's' of '-es' or '-ies'
        
        # S-addition rule for regular nouns
        transitions["q_root"]['s'] = ("q_accept_pl", "+N+PL")
        
        # FIXED: Add transition for Y-replacement rule (e.g., tries) from the root.
        transitions["q_root"]['i'] = ("q_i_plural", "y")


        return transitions

    def analyze(self, word):
        """
        Processes a word through the FST and returns its analysis.
        """
        word = word.lower()
        current_state = "q_start"
        output = ""

        # Process the word character by character
        for char in word:
            if char not in self.transitions[current_state]:
                return "Invalid Word"
            
            next_state, out_char = self.transitions[current_state][char]
            
            if next_state is None: # Explicit invalid transition
                 return "Invalid Word"

            current_state = next_state
            output += out_char
        
        # After the loop, determine the final output based on the final state
        if current_state == "q_accept_pl":
            return output
        # If we end in a valid root-ending state, it's a singular noun
        elif current_state in ["q_root", "q_y_ending", "q_sibilant", "q_s_ending", "q_c_ending"]:
            return output + "+N+SG"
        else:
            # Ending in an intermediate state (like q_e_plural) means the word is incomplete
            return "Invalid Word"

# --- Example Usage ---
fst = NounFST()

words_to_test = ["fox", "foxes", "watch", "watches", "try", "tries", "bag", "bags", "boy", "boys", "bus", "buses", "foxs", "bagz"]

for word in words_to_test:
    analysis = fst.analyze(word)
    print(f"{word:<10} -> {analysis}")


fox        -> fox+N+SG
foxes      -> fox+N+PL
watch      -> watch+N+SG
watches    -> watch+N+PL
try        -> try+N+SG
tries      -> try+N+PL
bag        -> bag+N+SG
bags       -> bag+N+PL
boy        -> boy+N+SG
boys       -> boy+N+PL
bus        -> bu+N+PL
buses      -> Invalid Word
foxs       -> Invalid Word
bagz       -> bagz+N+SG


In [6]:
import string

class NounFST:
    """
    A Finite State Transducer (FST) to generate morphological features for English nouns.
    This FST handles three common pluralization rules:
    1. S-addition (e.g., bag -> bags)
    2. E-insertion after sibilants (e.g., fox -> foxes)
    3. Y-replacement (e.g., try -> tries)
    """

    def __init__(self):
        # Define the character sets for easier rule creation
        self.alphabet = set(string.ascii_lowercase)
        self.sibilants_1 = {'s', 'x', 'z'}
        self.sibilants_2_starters = {'c', 's'} # For 'ch' and 'sh'
        
        # All other non-special consonants and vowels. 'i' is handled specially.
        self.nonspecial_chars = self.alphabet - self.sibilants_1 - self.sibilants_2_starters - {'y', 'h', 'i'}

        # Define the states of our FST
        self.states = {
            "q_start",      # Initial state
            "q_root",       # Generic state for processing the root of a word
            "q_s_ending",   # State after encountering an 's' (could be 'sh' or plural)
            "q_c_ending",   # State after encountering a 'c' (could be 'ch')
            "q_y_ending",   # State after encountering a 'y'
            "q_sibilant",   # State for roots ending in a sibilant sound (s, x, z, ch, sh)
            "q_e_plural",   # State after seeing 'e' in an '-es' plural
            "q_i_plural",   # State after seeing 'i' in an '-ies' plural
            "q_accept_pl",  # Final accepting state for plurals
        }

        # The core of the FST: the transition table
        # Format: transitions[current_state][input_char] = (next_state, output_string)
        self.transitions = self._create_transitions()

    def _create_transitions(self):
        """Builds the transition dictionary."""
        transitions = {state: {} for state in self.states}
        
        # --- Transitions from q_start (and q_root, as they are similar) ---
        for state in ["q_start", "q_root"]:
            # Rule: Nonspecial characters keep us in the root state
            for char in self.nonspecial_chars:
                transitions[state][char] = ("q_root", char)
            # Rule: Sibilants move to the sibilant state
            for char in self.sibilants_1:
                transitions[state][char] = ("q_sibilant", char)
            # Rule: 'y' moves to the y-ending state
            transitions[state]['y'] = ("q_y_ending", 'y')
            # Rule: 'c' and 's' are special and need their own states to check for 'h'
            transitions[state]['c'] = ("q_c_ending", 'c')
            transitions[state]['s'] = ("q_s_ending", 's')

        # --- Transitions from specific ending states ---
        
        # State after seeing 'c'
        transitions["q_c_ending"]['h'] = ("q_sibilant", 'h') # Forms 'ch'
        for char in self.alphabet - {'h'}: # Any other character
             # FIXED: Changed self.transitions to the local 'transitions' dictionary
             transitions["q_c_ending"][char] = transitions["q_root"].get(char, (None, None))

        # State after seeing 's'
        transitions["q_s_ending"]['h'] = ("q_sibilant", 'h') # Forms 'sh'
        transitions["q_s_ending"]['e'] = ("q_e_plural", "") # Plural of s-ending root (bus -> buses)
        for char in self.alphabet - {'h', 'e'}: # Any other character
             # FIXED: Changed self.transitions to the local 'transitions' dictionary
             transitions["q_s_ending"][char] = transitions["q_root"].get(char, (None, None))

        # State for sibilant-ending roots (e.g., fox, watch)
        transitions["q_sibilant"]['e'] = ("q_e_plural", "") # E-insertion rule (fox -> foxes)

        # State for y-ending roots (e.g., try, boy)
        transitions["q_y_ending"]['i'] = ("q_i_plural", "y") # Y-replacement rule (tries)
        transitions["q_y_ending"]['s'] = ("q_accept_pl", "+N+PL") # Simple plural (boys)

        # States for handling plural endings
        transitions["q_i_plural"]['e'] = ("q_e_plural", "") # Middle of '-ies'
        transitions["q_e_plural"]['s'] = ("q_accept_pl", "+N+PL") # Final 's' of '-es' or '-ies'
        
        # S-addition rule for regular nouns
        transitions["q_root"]['s'] = ("q_accept_pl", "+N+PL")
        
        # FIXED: Add transition for Y-replacement rule (e.g., tries) from the root.
        transitions["q_root"]['i'] = ("q_i_plural", "y")


        return transitions

    def analyze(self, word):
        """
        Processes a word through the FST and returns its analysis.
        """
        word = word.lower()
        current_state = "q_start"
        output = ""

        # Process the word character by character
        for char in word:
            if char not in self.transitions[current_state]:
                return "Invalid Word"
            
            next_state, out_char = self.transitions[current_state][char]
            
            if next_state is None: # Explicit invalid transition
                 return "Invalid Word"

            current_state = next_state
            output += out_char
        
        # After the loop, determine the final output based on the final state
        if current_state == "q_accept_pl":
            return output
        # If we end in a valid root-ending state, it's a singular noun
        elif current_state in ["q_root", "q_y_ending", "q_sibilant", "q_s_ending", "q_c_ending"]:
            return output + "+N+SG"
        else:
            # Ending in an intermediate state (like q_e_plural) means the word is incomplete
            return "Invalid Word"

# --- Example Usage ---
fst = NounFST()

words_to_test = ["fox", "foxes", "watch", "watches", "try", "tries", "bag", "bags", "boy", "boys", "bus", "buses", "foxs", "bagz"]

for word in words_to_test:
    analysis = fst.analyze(word)
    print(f"{word:<10} -> {analysis}")


fox        -> fox+N+SG
foxes      -> fox+N+PL
watch      -> watch+N+SG
watches    -> watch+N+PL
try        -> try+N+SG
tries      -> try+N+PL
bag        -> bag+N+SG
bags       -> bag+N+PL
boy        -> boy+N+SG
boys       -> boy+N+PL
bus        -> bu+N+PL
buses      -> Invalid Word
foxs       -> Invalid Word
bagz       -> bagz+N+SG


In [8]:
"""
Finite State Transducer for English Noun Morphology

This script implements a simple rule-based Finite State Transducer (FST)
to perform morphological analysis on a given set of English nouns.
It identifies the root word and determines if it is singular or plural.
"""

class FiniteStateTransducer:
    """
    A Finite State Transducer that analyzes English nouns.

    This FST is designed to handle common pluralization rules in English.
    The "states" and "transition table" are not represented by a formal
    data structure like a dictionary, but are implicitly defined by the
    conditional logic (if/elif/else blocks) within the `transduce` method.

    - Initial State: The start of the `transduce` method.
    - Transition Logic: Each `if word.endswith(...)` check acts as a transition
      based on the suffix of the input word.
    - Final States: The `return` statements that produce the final output string
      (e.g., "fox+N+PL") or "Invalid Word".
    """

    def __init__(self):
        """
        Initializes the FiniteStateTransducer.
        The rules are hardcoded into the `transduce` method.
        """
        pass

    def transduce(self, word: str) -> str:
        """
        Analyzes a word and transduces it to its morphological form.

        The order of these rules is critical, as it follows a longest-match
        first principle (e.g., checking for 'ies' before 's').

        Args:
            word: The input word to be analyzed.

        Returns:
            A string with the morphological analysis or "Invalid Word".
        """
        # Rule 1: Handles plurals ending in 'ies' (e.g., tries -> try)
        if word.endswith('ies'):
            stem = word[:-3] + 'y'
            return f"{stem}+N+PL"

        # Rule 2: Handles plurals ending in 'es' for sibilant sounds
        # (e.g., foxes, buses, watches)
        if word.endswith('es'):
            stem = word[:-2]
            # Check if the stem ends in a character that requires 'es' for pluralization
            if stem.endswith(('s', 'x', 'z', 'ch', 'sh')):
                return f"{stem}+N+PL"
            else:
                # This case could be ambiguous (e.g. 'goes'), but for the given
                # noun examples, we can treat other '-es' endings as simple '-s' plurals.
                # For example, 'candidates' would be handled by the next rule.
                # We pass to the next rule.
                pass

        # Rule 3: Handles standard plurals ending in 's'
        if word.endswith('s'):
            stem = word[:-1]
            # INVALID WORD CHECK: A word ending in 'x' or 's' cannot be pluralized
            # with just an 's'; it requires 'es'.
            if stem.endswith(('x', 's', 'z', 'ch', 'sh')):
                return "Invalid Word"
            
            # This is a standard plural (e.g., candidates -> candidate)
            return f"{stem}+N+PL"

        # Rule 4: If no plural rule matches, assume it's a singular noun.
        # This is the default case.
        return f"{word}+N+SG"

# --- Main execution block to demonstrate the FST ---
if __name__ == "__main__":
    # Create an instance of our transducer
    fst = FiniteStateTransducer()

    # List of words to test
    test_words = [
        "fox",
        "foxes",
        "foxs",  # Expected: Invalid
        "candidate",
        "candidates",
        "bus",
        "buses",
        "try",
        "tries",
        "watch",
        "watches",
        "cat",
        "cats",
        "potato", # Note: Doesn't handle 'potatoes' correctly, needs a more complex rule
        "potatos" # Expected: Invalid based on our simple rules
    ]

    print("--- FST Morphological Analysis ---")
    print("-" * 34)

    # Process each word and print the result
    for word in test_words:
        result = fst.transduce(word)
        # The f-string formatting helps align the output for readability
        print(f"Input: {word:<12} -> Output: {result}")

    print("-" * 34)



--- FST Morphological Analysis ---
----------------------------------
Input: fox          -> Output: fox+N+SG
Input: foxes        -> Output: fox+N+PL
Input: foxs         -> Output: Invalid Word
Input: candidate    -> Output: candidate+N+SG
Input: candidates   -> Output: candidate+N+PL
Input: bus          -> Output: bu+N+PL
Input: buses        -> Output: bus+N+PL
Input: try          -> Output: try+N+SG
Input: tries        -> Output: try+N+PL
Input: watch        -> Output: watch+N+SG
Input: watches      -> Output: watch+N+PL
Input: cat          -> Output: cat+N+SG
Input: cats         -> Output: cat+N+PL
Input: potato       -> Output: potato+N+SG
Input: potatos      -> Output: potato+N+PL
----------------------------------


In [15]:
# FST for detecting SG/PL from brown_nouns.txt
# Reads nouns from file and identifies root + N + SG/PL
# Saves output to out.txt

def load_nouns(file_path):
    """Load all nouns from the file into a set."""
    with open(file_path, 'r') as f:
        return set(word.strip().lower() for word in f if word.strip())

def is_vowel(ch):
    return ch in "aeiou"

def detect_number(word, noun_set):
    """Return root+N+SG or root+N+PL if valid, else 'Invalid Word'."""
    w = word.lower()

    # Must exist in noun list
    if w not in noun_set:
        return "Invalid Word"

    # --- Detect Plural Forms ---
    # E insertion: matches foxes, watches, etc.
    if w.endswith("es") and (
        w[-3] in ["s", "z", "x"] or w.endswith(("ches", "shes"))
    ):
        root = w[:-2]
        return f"{root}+N+PL"

    # Y replacement: matches tries, flies, etc.
    if w.endswith("ies") and len(w) > 3:
        root = w[:-3] + "y"
        return f"{root}+N+PL"

    # Default plural: ends with 's' (bags, cats, etc.)
    if w.endswith("s") and len(w) > 1:
        root = w[:-1]
        return f"{root}+N+PL"

    # --- If not plural, treat as singular ---
    return f"{w}+N+SG"

if __name__ == "__main__":
    noun_set = load_nouns("brown_nouns.txt")

    with open("out2.txt", "w") as out_file:
        for noun in noun_set:
            result = detect_number(noun, noun_set)
            out_file.write(f"{noun} -> {result}\n")


# Finite State Transducer (FST) for English Noun Pluralization

## 1. Input & Output Alphabets

**Input Alphabet Σ:**  
- All lowercase English letters `a-z`  
- Special morphological markers: `{N}`, `{SG}`, `{PL}`

**Output Alphabet Γ:**  
- All lowercase English letters `a-z`  
- Morphological tags: `+N`, `+SG`, `+PL`

---

## 2. States

| State | Meaning                                      |
|-------|----------------------------------------------|
| q0    | Start state                                  |
| q1    | Reading root letters (default path)          |
| q2    | Last letter was `y` (possible Y replacement) |
| q3    | Last letter was `s`, `z`, `x`, `ch`, or `sh` (possible E insertion) |
| q4    | Adding plural suffix                         |
| q5    | Accept state (word recognized)               |
| qerr  | Error state (Invalid Word)                   |

---

## 3. Transition Table

| Current State | Input Symbol                  | Output Symbol | Next State | Notes                                       |
|---------------|------------------------------|--------------|------------|---------------------------------------------|
| q0            | letter (not y, s, z, x, h)   | same         | q1         | Normal root                                 |
| q0            | y                            | y            | q2         | Potential Y replacement                     |
| q0            | s/z/x/h(ch/sh)               | same         | q3         | Potential E insertion                       |
| q1            | letter (not y, s, z, x, h)   | same         | q1         | Continue root                               |
| q1            | y                            | y            | q2         | Y replacement possible                      |
| q1            | s/z/x/h(ch/sh)               | same         | q3         | E insertion possible                        |
| q2            | {PL}                         | ie+s         | q5         | Y → ie + s                                  |
| q2            | {SG}                         | y            | q5         | Singular form stays same                    |
| q3            | {PL}                         | e+s          | q5         | E insertion rule                            |
| q3            | {SG}                         | same         | q5         | Singular form                               |
| q1            | {PL}                         | s            | q5         | Normal S addition                           |
| q5            | —                            | —            | q5         | Final state                                 |
| any           | invalid char                 | —            | qerr       | Invalid Word                                |

---

## 4. How Rules Work in FST

### Rule 1 — E Insertion
If the final character in the root is in `{s, z, x}` or ends in `ch`/`sh`, insert `e` before `s`.

> **Example:**  
> Input: `f o x {PL}`  
> Output: `f o x e s` → `fox+N+PL`

---

### Rule 2 — Y Replacement
If the final character is `y` preceded by a consonant, replace `y` with `ie` before adding `s`.

> **Example:**  
> Input: `t r y {PL}`  
> Output: `t r i e s` → `try+N+PL`

---

### Rule 3 — S Addition
Default: append `s`.

> **Example:**  
> Input: `b a g {PL}`  
> Output: `b a g s` → `bag+N+PL`

---

## 5. Invalid Word Handling

If a word does **not** match any morphological rule or is **not** in `brown_nouns.txt` → transition to `qerr` and output `"Invalid Word"`.

> **Example:**  
> Input: `f o x s`  
> Not matching E-insertion path → `qerr` → `"Invalid Word"`

---

## 6. Example Transductions

| Input             | Output          | Final Form      |
|-------------------|----------------|-----------------|
| `f o x {PL}`      | `f o x e s`    | `fox+N+PL`      |
| `t r y {PL}`      | `t r i e s`    | `try+N+PL`      |
| `b a g {PL}`      | `b a g s`      | `bag+N+PL`      |
| `f o x s`         | Invalid Word   | (qerr)          |

---

## 7. State Reduction Technique (Bonus Marks)

- Merge `q1`, `q2`, `q3` by **classifying the last letter** in transitions.
- Store the "last letter type" (normal, y, special-e) as a feature rather than separate states.
- On `{PL}`, decide output based on last-letter classification.
- Simplifies the FST and reduces number of unique states.

---

### **Summary Diagram**

```
(Start) q0
   ↓
  [read letters]
   ↓
  q1 ──> (last letter = y) ──> q2
   │                          │
   │                      {PL}: y→ie+s
   │
   └─> (last letter in {s,z,x,ch,sh}) ──> q3
                                    │
                                {PL}: e+s
   |
   └─> {PL}: s (default)
```

---

In [14]:
# FST for detecting SG/PL from brown_nouns.txt with state tracking and transition table output

def load_nouns(file_path):
    """Load all nouns from the file into a set."""
    with open(file_path, 'r') as f:
        return set(word.strip().lower() for word in f if word.strip())

def is_vowel(ch):
    return ch in "aeiou"

def detect_number_with_states(word, noun_set):
    """
    Return (result, state_trace, transition_trace).
    result: root+N+SG or root+N+PL if valid, else 'Invalid Word'.
    state_trace: list of (state, input_symbol, output_symbol, next_state).
    transition_trace: summary table of transitions taken.
    """
    w = word.lower()
    state_trace = []
    transitions = []

    # Start at q0
    state = 'q0'

    # Must exist in noun list
    if w not in noun_set:
        state_trace.append((state, w, 'Invalid Word', 'qerr'))
        transitions.append({
            'Current State': state,
            'Input Symbol': w,
            'Output Symbol': 'Invalid Word',
            'Next State': 'qerr',
            'Notes': 'Word not in brown_nouns.txt'
        })
        return "Invalid Word", state_trace, transitions

    orig_w = w

    # E insertion: foxes, watches, etc.
    if w.endswith("es") and (
        (len(w)>=3 and w[-3] in ["s", "z", "x"])
        or w.endswith("ches") or w.endswith("shes")
    ):
        # q0 --[letters]--> q3 --["es"]--> q5
        for i in range(len(w)-2):  # root part
            state_trace.append((state, w[i], w[i], 'q3'))
            transitions.append({
                'Current State': state,
                'Input Symbol': w[i],
                'Output Symbol': w[i],
                'Next State': 'q3',
                'Notes': 'Root letter'
            })
            state = 'q3'
        state_trace.append((state, w[-2:], 'e+s', 'q5'))
        transitions.append({
            'Current State': state,
            'Input Symbol': w[-2:],
            'Output Symbol': 'e+s',
            'Next State': 'q5',
            'Notes': 'E-insertion plural'
        })
        root = w[:-2]
        return f"{root}+N+PL", state_trace, transitions

    # Y replacement: tries, flies, etc.
    if w.endswith("ies") and len(w) > 3:
        # q0 --[letters]--> q2 --["ies"]--> q5
        for i in range(len(w)-3):  # root part
            state_trace.append((state, w[i], w[i], 'q2'))
            transitions.append({
                'Current State': state,
                'Input Symbol': w[i],
                'Output Symbol': w[i],
                'Next State': 'q2',
                'Notes': 'Root letter'
            })
            state = 'q2'
        state_trace.append((state, w[-3:], 'ie+s', 'q5'))
        transitions.append({
            'Current State': state,
            'Input Symbol': w[-3:],
            'Output Symbol': 'ie+s',
            'Next State': 'q5',
            'Notes': 'Y-replacement plural'
        })
        root = w[:-3] + "y"
        return f"{root}+N+PL", state_trace, transitions

    # Default plural: ends with 's' (bags, cats, etc.)
    if w.endswith("s") and len(w) > 1:
        # q0 --[letters]--> q1 --["s"]--> q5
        for i in range(len(w)-1):  # root part
            state_trace.append((state, w[i], w[i], 'q1'))
            transitions.append({
                'Current State': state,
                'Input Symbol': w[i],
                'Output Symbol': w[i],
                'Next State': 'q1',
                'Notes': 'Root letter'
            })
            state = 'q1'
        state_trace.append((state, w[-1], 's', 'q5'))
        transitions.append({
            'Current State': state,
            'Input Symbol': w[-1],
            'Output Symbol': 's',
            'Next State': 'q5',
            'Notes': 'Default plural'
        })
        root = w[:-1]
        return f"{root}+N+PL", state_trace, transitions

    # Singular: treat as singular, no plural ending
    # q0 --[letters]--> q1 --(end)--> q5
    for i in range(len(w)):  # root part
        state_trace.append((state, w[i], w[i], 'q1'))
        transitions.append({
            'Current State': state,
            'Input Symbol': w[i],
            'Output Symbol': w[i],
            'Next State': 'q1',
            'Notes': 'Root letter'
        })
        state = 'q1'
    state_trace.append((state, '(end)', '+SG', 'q5'))
    transitions.append({
        'Current State': state,
        'Input Symbol': '(end)',
        'Output Symbol': '+SG',
        'Next State': 'q5',
        'Notes': 'Singular'
    })
    return f"{w}+N+SG", state_trace, transitions

def print_transition_table(transitions):
    print("\nTransition Table (Executed):")
    print("{:<12} {:<12} {:<14} {:<10} {:<25}".format(
        'Current St', 'Input Sym', 'Output Sym', 'Next St', 'Notes'))
    print('-'*75)
    for t in transitions:
        print("{:<12} {:<12} {:<14} {:<10} {:<25}".format(
            t['Current State'], t['Input Symbol'], t['Output Symbol'], t['Next State'], t['Notes']))

if __name__ == "__main__":
    noun_set = load_nouns("brown_nouns.txt")

    with open("out.txt", "w") as out_file:
        for noun in noun_set:
            result, state_trace, transitions = detect_number_with_states(noun, noun_set)
            out_file.write(f"{noun} -> {result}\n")
            out_file.write("States:\n")
            for (state, symbol, output, next_state) in state_trace:
                out_file.write(f"  ({state}, {symbol}) -> ({output}) -> {next_state}\n")
            out_file.write("Transition Table:\n")
            out_file.write("{:<12} {:<12} {:<14} {:<10} {:<25}\n".format(
                'Current St', 'Input Sym', 'Output Sym', 'Next St', 'Notes'))
            out_file.write('-'*75+'\n')
            for t in transitions:
                out_file.write("{:<12} {:<12} {:<14} {:<10} {:<25}\n".format(
                    t['Current State'], t['Input Symbol'], t['Output Symbol'], t['Next State'], t['Notes']))
            out_file.write('\n')